In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image

from model import *
from train import *

import os
import shutil
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')

device = cpu


# Load pretrained automaton

In [7]:
name = 'bubbles_bias_order012_mean_8000ep'

automaton = CAutomaton(bias=True)
automaton.load_state_dict(torch.load(f'pretrained_automata/{name}_state_dict.pt', map_location=device))
#set_perception_kernels(automaton, angle=np.pi/2)

print(automaton)

CAutomaton(
  (perception_filter): Conv2d(12, 48, kernel_size=(3, 3), stride=(1, 1))
  (update_rule): Sequential(
    (0): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(96, 12, kernel_size=(1, 1), stride=(1, 1))
  )
)


In [8]:
automaton.update_rule[2].bias

Parameter containing:
tensor([-0.0025, -0.0026,  0.0033, -0.0005, -0.0010,  0.0006,  0.0199, -0.0014,
         0.0030,  0.0087, -0.0038, -0.0080], requires_grad=True)

In [ ]:
# load loss data

with open(f'pretrained_automata/{name}_tlosses.pickle', 'rb') as f:
    texture_losses = pickle.load(f)
with open(f'pretrained_automata/{name}_dlosses.pickle', 'rb') as f:
    domain_losses = pickle.load(f)
with open(f'pretrained_automata/{name}_losses.pickle', 'rb') as f:
    losses = pickle.load(f)

# plot evolution

plt.semilogy(losses, ',', color='C2', label='total loss')
plt.semilogy(texture_losses, ',', color='C0', label='texture loss')
plt.semilogy(domain_losses, ',', color='C1', label='domain_loss')
plt.legend()
plt.show()

plt.plot(losses[200:], '.', color='C2', label='total loss')
plt.plot(texture_losses[200:], '.', color='C0', label='texture loss')
plt.plot(domain_losses[200:], '.', color='C1', label='domain_loss')
plt.legend()
plt.show()

# Test automaton

In [9]:
size = (128, 128)

from torchvision.utils import save_image

if os.path.exists('outputs'):
    shutil.rmtree('outputs')
    os.mkdir('outputs')
else:
    os.mkdir('outputs')

test_iters = 500

# iterate automaton from random initial state
with torch.inference_mode():
    #states = torch.rand((1, automaton.num_states, *size), device=device)
    states = 0.5*torch.ones((1, automaton.num_states, *size), device=device) + 0.05*torch.randn((1, automaton.num_states, *size), device=device)
    for step in range(test_iters):
        states = automaton(states)
        img = states[:, :3, :, :]

        save_image(img[0], f'outputs/epoch_{step:05}.png')


In [6]:
# Make a gif animation
def make_gif(test_iters, name):
    frames = [Image.open(f'outputs/epoch_{iter:05}.png') for iter in range(test_iters)]
    frame_one = frames[0]
    frame_one.save(f"gifs/{name}_turinginit_{size[0]}p_{test_iters}iter_evolution.gif", format="GIF", append_images=frames,
               save_all=True, duration=10, loop=0)

make_gif(test_iters, name)

## First order terms?

In [ ]:
n = 256

x = torch.linspace(0, 1, n)[None, None, :, None].expand((1, 12, n, n))
y = torch.linspace(0, 1, n)[None, None, None, :].expand((1, 12, n, n))

for i in range(50):
    wx, wy, p = torch.rand((12)), torch.rand((12)), torch.rand((12))
    wx = (wx/wx.sum())[None, :, None, None].expand((1, 12, n, n))
    wy = (wy/wy.sum())[None, :, None, None].expand((1, 12, n, n))
    p = p[None, :, None, None].expand((1, 12, n, n))
    states = p*wx*x+(1-p)*wy*y
    print((automaton(states)-states).abs().max().item())
    #print((automaton(states)-states).abs().mean().item())
